# **Set up**

In [1]:
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00


In [3]:
from openai import OpenAI

client = OpenAI(
  api_key='YOUR_API_KEY',
)

In [4]:
def get_gpt_response(prompt, max_tokens=200):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        # max_tokens=500,
        # n=1,
        # stop="\n",  # Set a stop sequence here
        # temperature=0.7,
    )
    return response.choices[0].message.content

In [54]:
import torch
import torch.nn.functional as F

# **Prompting**

**Require:** training samples and score function

In [5]:
words = ["lành mạnh", "trực tiếp", "phi chính thức", "không phổ biến", "trừ đi", "không ở lại",
"không chính xác", "phía trên thành phố", "không thể so sánh", "mạnh mẽ", "khí", "đồng đều", "nghi thức",
"cố ý", "tắt"]
antonyms = ["điên", "gián tiếp", "chính thức", "phổ biến", "cộng", "ở lại",
"chính xác", "phố phường", "có thể so sánh", "yếu", "rắn", "không đều", "phi chính thức",
"ngẫu nhiên", "bật"]

In [16]:
train_data = [(word, antonym) for word, antonym in zip(words, antonyms)]

In [17]:
train_data

[('lành mạnh', 'điên'),
 ('trực tiếp', 'gián tiếp'),
 ('phi chính thức', 'chính thức'),
 ('không phổ biến', 'phổ biến'),
 ('trừ đi', 'cộng'),
 ('không ở lại', 'ở lại'),
 ('không chính xác', 'chính xác'),
 ('phía trên thành phố', 'phố phường'),
 ('không thể so sánh', 'có thể so sánh'),
 ('mạnh mẽ', 'yếu'),
 ('khí', 'rắn'),
 ('đồng đều', 'không đều'),
 ('nghi thức', 'phi chính thức'),
 ('cố ý', 'ngẫu nhiên'),
 ('tắt', 'bật')]

In [49]:
def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [55]:
def score_function(text1, text2):
  vector1 = torch.tensor(get_embedding(text1))
  vector2 = torch.tensor(get_embedding(text2))

  # Tính cosine similarity
  similarity = F.cosine_similarity(vector1.unsqueeze(0), vector2.unsqueeze(0))

  return similarity.item()

**Bước 1:** Sử dụng một mô hình LLMs để tạo sinh ra một tập instruction đề xuất.

In [8]:
instructions = []

llm_inference_template = '''
Tôi đưa cho bạn tôi một chỉ dẫn và đây là câu trả lời của bạn tôi
"Input: vui - Output: buồn"
"Input: thành công - Output: thất bại"
"Input: tốt - Output: xấu"

Chỉ dẫn của tôi dùng để
'''

In [9]:
for i in range(10):
  response = get_gpt_response(llm_inference_template)
  instructions.append(response)
  print('{}: {}'.format(i, response))

0: đảo ngược các từ và tạo ra các từ trái nghĩa.
1: đảo ngược ý nghĩa của các từ được nhập vào.
2:  đảo ngược ý nghĩa của từ, từ "Input" sang "Output".
3: đảo ngược ý nghĩa của từ đầu vào thành từ đầu ra.
4: chuyển đổi từ các từ tích cực sang tiêu cực và ngược lại.
5:  đảo ngược ý nghĩa của các từ trong câu.
6: đảo ngược nghĩa của các từ trong câu nhập để tạo ra câu trả lời.
7: đảo ngược các từ trong ví dụ cho trước, chuyển từ tích cực sang tiêu cực và ngược lại.
8: đảo ngược ý nghĩa của từ đầu vào để tạo ra từ đầu ra tương phản.
9: đảo ngược ý nghĩa của từ đầu vào để tạo ra từ đầu ra tương tự như từ đầu vào nhưng mang ý nghĩa ngược lại.


**Vậy các instruction mà LLMs để xuất gồm:**

0: đảo ngược các từ và tạo ra các từ trái nghĩa. <br>
1: đảo ngược ý nghĩa của các từ được nhập vào. <br>
2:  đảo ngược ý nghĩa của từ, từ "Input" sang "Output". <br>
3: đảo ngược ý nghĩa của từ đầu vào thành từ đầu ra. <br>
4: chuyển đổi từ các từ tích cực sang tiêu cực và ngược lại. <br>
5:  đảo ngược ý nghĩa của các từ trong câu. <br>
6: đảo ngược nghĩa của các từ trong câu nhập để tạo ra câu trả lời. <br>
7: đảo ngược các từ trong ví dụ cho trước, chuyển từ tích cực sang tiêu cực và ngược lại. <br>
8: đảo ngược ý nghĩa của từ đầu vào để tạo ra từ đầu ra tương phản. <br>
9: đảo ngược ý nghĩa của từ đầu vào để tạo ra từ đầu ra tương tự như từ đầu vào nhưng mang ý nghĩa ngược lại.

**Bước 2:** Chọn một subset từ tập train, sau đó đánh giá score của từng prompt đề xuất trên subset đó.

In [23]:
import random

# Lấy ngẫu nhiên 10 cặp từ
subset = random.sample(train_data, 10)
subset

[('cố ý', 'ngẫu nhiên'),
 ('đồng đều', 'không đều'),
 ('không thể so sánh', 'có thể so sánh'),
 ('không phổ biến', 'phổ biến'),
 ('không ở lại', 'ở lại'),
 ('nghi thức', 'phi chính thức'),
 ('khí', 'rắn'),
 ('trực tiếp', 'gián tiếp'),
 ('phi chính thức', 'chính thức'),
 ('lành mạnh', 'điên')]

In [39]:
llm_scores_template = '''
Instruction: {}
Input: {}
Output:
'''

In [43]:
llm_words_output = []

for instruct in instructions:
  word_with_instruct = []
  for w in subset:
    response = get_gpt_response(llm_scores_template.format(instruct, w[0]))
    word_with_instruct.append(response)
  llm_words_output.append(word_with_instruct)

In [46]:
# Output với các instructions tương ứng
for i in llm_words_output:
  print(i)

['Input: cố ý\nOutput: ý cố\n\nTừ trái nghĩa: vô tình', 'Input: không đều\nOutput: đồng đều', 'không thể nghĩ so\nnon không thể so', 'không nềb ổhp\nnghị pảnậy', 'không đi xa', 'Input: nghi thức\nOutput: ịưhgn', 'Input: đẹp\nOutput: xấu', 'đính ngược', 'Input: an chính phủ\nOutput: không phương thức', 'Input: lành mạnh\nOutput: hạnh mạnh, xấu xí']
['không cố ý', 'bất đồng', 'có thể so sánh', 'phổ biến', 'ở lại không', 'thức nghi', 'sức', 'gián tiếp', 'chính thức phi', 'độc hại']
['vô ý', 'không đồng đều', 'có thể so sánh', 'phổ biến', 'ở lại không', 'Output: không thức', 'Output: không', 'gián tiếp', 'Output: chính thức phi', 'độc hại']
['vô ý', 'bất đồng', 'có thể so sánh', 'phổ biến', 'ở lại không', 'không thức', 'không', 'gián tiếp', 'chính thức đúng', 'ốnh lãnh']
['Input: không biết\nOutput: biết choàng', 'Input: thiếu điểm\nOutput: điều thành công', 'Không thể không so sánh', 'phổ biến', 'Input: không rời đi\nOutput: ở lại', 'Input: tiện ích\nOutput: không tiện ích', 'Input: trời\

In [63]:
# Tính score giữa words và output của các instructions tương ứng
scores = []

for i in llm_words_output:
  score = 0
  for w, sbs in zip(i, subset):
    score += score_function(w, sbs[1])

  scores.append(score/10)

In [64]:
print(scores)

[0.38801275193691254, 0.6785409331321717, 0.6610703051090241, 0.6705738067626953, 0.49382856786251067, 0.583047142624855, 0.628467458486557, 0.5129027128219604, 0.6466999471187591, 0.6917043060064316]


In [67]:
instructions_scores = dict(zip(instructions, scores))

In [70]:
for i in instructions_scores.items():
  print(i)

('đảo ngược các từ và tạo ra các từ trái nghĩa.', 0.38801275193691254)
('đảo ngược ý nghĩa của các từ được nhập vào.', 0.6785409331321717)
(' đảo ngược ý nghĩa của từ, từ "Input" sang "Output".', 0.6610703051090241)
('đảo ngược ý nghĩa của từ đầu vào thành từ đầu ra.', 0.6705738067626953)
('chuyển đổi từ các từ tích cực sang tiêu cực và ngược lại.', 0.49382856786251067)
(' đảo ngược ý nghĩa của các từ trong câu.', 0.583047142624855)
('đảo ngược nghĩa của các từ trong câu nhập để tạo ra câu trả lời.', 0.628467458486557)
('đảo ngược các từ trong ví dụ cho trước, chuyển từ tích cực sang tiêu cực và ngược lại.', 0.5129027128219604)
('đảo ngược ý nghĩa của từ đầu vào để tạo ra từ đầu ra tương phản.', 0.6466999471187591)
('đảo ngược ý nghĩa của từ đầu vào để tạo ra từ đầu ra tương tự như từ đầu vào nhưng mang ý nghĩa ngược lại.', 0.6917043060064316)


**Bước 3:** Lọc ra top k prompt có kết quả cao nhất

In [77]:
sorted_scores = sorted(instructions_scores.items(), key=lambda x: x[1],  reverse=True)
# Lọc ra top 5
final_scores = sorted_scores[:5]
final_scores

[('đảo ngược ý nghĩa của từ đầu vào để tạo ra từ đầu ra tương tự như từ đầu vào nhưng mang ý nghĩa ngược lại.',
  0.6917043060064316),
 ('đảo ngược ý nghĩa của các từ được nhập vào.', 0.6785409331321717),
 ('đảo ngược ý nghĩa của từ đầu vào thành từ đầu ra.', 0.6705738067626953),
 (' đảo ngược ý nghĩa của từ, từ "Input" sang "Output".', 0.6610703051090241),
 ('đảo ngược ý nghĩa của từ đầu vào để tạo ra từ đầu ra tương phản.',
  0.6466999471187591)]

**Optional:** Sử dụng mô hình LLM để resample lại

In [87]:
llm_resample_template = '''
Instruction: Tạo ra một biến thể của hướng dẫn sau trong khi vẫn giữ được ngữ nghĩa của câu.
Input: đảo ngược ý nghĩa của từ đầu vào để tạo ra từ đầu ra tương tự như từ đầu vào nhưng mang ý nghĩa ngược lại.
Output:
'''

In [88]:
resample_instruction = get_gpt_response(llm_resample_template)
resample_instruction

'Đảo ngược ý nghĩa của từ đầu vào sẽ tạo ra từ đầu ra trái ngược với từ ban đầu nhưng vẫn giữ nguyên ý nghĩa.'

In [89]:
# Tìm từ được sinh ra với instruction đã được resample trên tập subset
word_from_resample_instruction = []
for w in subset:
    response = get_gpt_response(llm_scores_template.format(resample_instruction, w[0]))
    word_from_resample_instruction.append(response)

word_from_resample_instruction

['vô ý',
 'không đều',
 'có thể đối chiếu',
 'phổ biến',
 'rời đi',
 'không thức',
 'chất',
 'gián tiếp',
 'chính thức phi',
 'ốn đau']

In [91]:
# Tính lại score với resample instruction
resample_score = 0
for w, sbs in zip(word_from_resample_instruction, subset):
    resample_score += score_function(w, sbs[1])
resample_score = resample_score/10
print(resample_score)

0.6526825070381165
